In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
import gradio as gr
import json

History is:
[{'role': 'user', 'metadata': None, 'content': 'Hi there', 'options': None}]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'metadata': None, 'content': 'Hi there', 'options': None}, {'role': 'user', 'content': 'Hi there'}]


In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyB2


In [3]:
# Initialize

# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

- Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [19]:
system_message = "You are a helpful assistant"

In [23]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [24]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [25]:
# code example to make requests to ollama
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as blog posts, articles, social media posts, product descriptions, and more. This can help reduce content creation costs and improve content consistency.
2. **Marketing Automation**: Generative AI can be used to personalize marketing messages, create targeted advertising campaigns, and automate social media posting.
3. **Product Design**: Generative AI can design new products, logos, and packaging designs, reducing the need for human designers and accelerating the product development process.
4. **Customer Service**: Generative AI-powered chatbots can provide 24/7 customer support, answering frequent questions and routing complex issues to human representatives.
5. **Predictive Maintenance**: Generative AI can analyze sensor data from machines and predict when maintenance is needed, reducing downtime and increasing equipme

In [32]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": True
    }

    response = ""
    # stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
    stream = requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True)

    for line in stream.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8'))
                if data.get("done"):
                    break
                delta = data.get("message", {}).get("content", "")
                if delta:
                    response += delta
                    yield response
            except Exception as e:
                print("Error parsing stream:", e)

    if not response:
        yield "No response received."

In [33]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hi there'}]
History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hi there'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'Hi there', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "How can I assist you today? Do you have any questions or topics you'd like to discuss? I'm all ears!", 'options': None}]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'metadata': None, 'content': 'Hi there', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "How can I assist you today? Do you have any questions or topics you'd like to discuss? I'm all ears!", 'options': None}, {'role': 'user', 'content': 'Can you tell me a joke?'}]
